# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Problema 2
#### Aluno: Eduardo Cesar Cauduro Coelho
#### Matrícula: 202310175
#### Aluno: Felipe Geraldo de Oliveira
#### Matrícula: 202310174

## Instalação e importação da biblioteca PuLP


In [21]:
!pip install pulp
import pulp as pl

## Declaração dos parâmetros

* $S$: conjunto de semanas.     
* $C_s \in \mathbb{R}_+$: custo de produção de cada camiseta por semana $s \in S$.      
* $D_s \in \mathbb{Z}_+$: demanda de camisetas por semana $s \in S$.
* $c \in \mathbb{Z}_+$: capacidade de produção de camisetas por semana.  
* $E_s \in \mathbb{Z}_+$: capacidade de produção extra de camisetas por semana $s \in S$.
* $CE_s \in \mathbb{R}_+$: custo de produção de cada camiseta extra por semana $s \in S$.
* $cs \in \mathbb{R}_+$: custo de estoque de cada camiseta.



In [22]:
semanas = ["semana_1", "semana_2", "semana_3", "semana_4"]

custo_por_semana = {
    "semana_1": 2.0,
    "semana_2": 2.0,
    "semana_3": 2.5,
    "semana_4": 2.5
}

demanda_por_semana = {
    "semana_1": 5000,
    "semana_2": 10000,
    "semana_3": 30000,
    "semana_4": 60000
}

capacidade_producao = 25000

capacidade_producao_extra_semana = {
    "semana_1": 10000,
    "semana_2": 10000,
    "semana_3": 0,
    "semana_4": 0
}

custo_producao_extra = 2.8

custo_estoque = 0.5

## Declaração do objeto que representa o modelo matemático

In [23]:
model = pl.LpProblem("minimo_custo", pl.LpMinimize)

## Variáveis de decisão

* $p_s \in \mathbb{Z}_+$: camisetas produzidas em cada semana $s \in S$.
* $e_s \in \mathbb{Z}_+$: camisetas extra produzidas em cada semana $s \in S$.
* $t_s \in \mathbb{Z}_+$: camisetas estocadas em cada semana $s \in S$.

In [24]:
camisetas_produzidas = pl.LpVariable.dicts("camisetas_produzidas", [semana for semana in semanas],  lowBound=0, cat="Integer")
camisetas_produzidas_extra = pl.LpVariable.dicts("camisetas_produzidas_extra", [semana for semana in semanas], lowBound=0, cat="Integer")
camisetas_estocadas = pl.LpVariable.dicts("camisetas_estocadas", [semana for semana in semanas],  lowBound=0, cat="Integer")

## Função objetivo

* Minimização do custo envolvendo produção e estoque de camisetas:\begin{equation}\min (\sum \limits_{s \in S}(p_s \times C_s) + \sum \limits_{s \in S}(e_s \times CE_s) + \sum \limits_{s \in S}(t_s \times cs))\end{equation}

In [25]:
model += pl.lpSum([custo_por_semana[semana] * camisetas_produzidas[semana]
                    + custo_producao_extra * camisetas_produzidas_extra[semana]
                    + custo_estoque  * camisetas_estocadas[semana] for semana in semanas])

## Restrições

* O numero de camisetas produzidas e previamente estocadas ou são consumidas na demanda ou vão para/continuam no estoque: \begin{equation}p_s + e_s + t_{s-1} - D_s - t_s = 0, \forall s \in S\end{equation} onde $s-1$ representa a semana anterior, e, portanto, na primeira semana $t_{s-1} = 0$.

In [26]:
for semana in semanas:
    if semana != "semana_1":
        semana_anterior = semana[:-1]
        numero_anterior = int(semana[-1]) - 1
        semana_anterior = semana_anterior + str(numero_anterior)
        model += camisetas_produzidas[semana] + camisetas_produzidas_extra[semana] - demanda_por_semana[semana] - camisetas_estocadas[semana] + camisetas_estocadas[semana_anterior] == 0
    else:
        model += camisetas_produzidas[semana] + camisetas_produzidas_extra[semana] - demanda_por_semana[semana] - camisetas_estocadas[semana] == 0

* O número de camisetas produzidas por semana não pode ultrapassar a capacidade de produção base: \begin{equation}p_s \le c, \forall s \in S\end{equation}
* O número de camisetas extra produzidas não pode ultrapassar a capacidade de produção extra: \begin{equation}e_s \le E_s, \forall s \in S\end{equation}

In [27]:
for semana in semanas:
    model += camisetas_produzidas[semana] <= capacidade_producao
    model += camisetas_produzidas_extra[semana] <= capacidade_producao_extra_semana[semana]

## Apresentação do modelo

In [28]:
model

minimo_custo:
MINIMIZE
0.5*camisetas_estocadas_semana_1 + 0.5*camisetas_estocadas_semana_2 + 0.5*camisetas_estocadas_semana_3 + 0.5*camisetas_estocadas_semana_4 + 2.8*camisetas_produzidas_extra_semana_1 + 2.8*camisetas_produzidas_extra_semana_2 + 2.8*camisetas_produzidas_extra_semana_3 + 2.8*camisetas_produzidas_extra_semana_4 + 2.0*camisetas_produzidas_semana_1 + 2.0*camisetas_produzidas_semana_2 + 2.5*camisetas_produzidas_semana_3 + 2.5*camisetas_produzidas_semana_4 + 0.0
SUBJECT TO
_C1: - camisetas_estocadas_semana_1 + camisetas_produzidas_extra_semana_1
 + camisetas_produzidas_semana_1 = 5000

_C2: camisetas_estocadas_semana_1 - camisetas_estocadas_semana_2
 + camisetas_produzidas_extra_semana_2 + camisetas_produzidas_semana_2 = 10000

_C3: camisetas_estocadas_semana_2 - camisetas_estocadas_semana_3
 + camisetas_produzidas_extra_semana_3 + camisetas_produzidas_semana_3 = 30000

_C4: camisetas_estocadas_semana_3 - camisetas_estocadas_semana_4
 + camisetas_produzidas_extra_semana_4 +

## Resolvendo o problema

In [29]:
status = model.solve()

## Impressão das soluções do problema

In [30]:
print('status: ', pl.LpStatus[status])
print('funcao objetivo: ', model.objective.value())
print('solucoes')
for semana in semanas:
    print(semana,
          f'Camisetas Produzidas: {camisetas_produzidas[semana].value()}',
          f'Camisetas Produzidas Extra: {camisetas_produzidas_extra[semana].value()}',
          f'Camisetas Estocadas: {camisetas_estocadas[semana].value()}',
          sep='\n', end='\n\n\n')

status:  Optimal
funcao objetivo:  286500.0
solucoes
semana_1
Camisetas Produzidas: 25000.0
Camisetas Produzidas Extra: 0.0
Camisetas Estocadas: 20000.0


semana_2
Camisetas Produzidas: 25000.0
Camisetas Produzidas Extra: 5000.0
Camisetas Estocadas: 40000.0


semana_3
Camisetas Produzidas: 25000.0
Camisetas Produzidas Extra: 0.0
Camisetas Estocadas: 35000.0


semana_4
Camisetas Produzidas: 25000.0
Camisetas Produzidas Extra: 0.0
Camisetas Estocadas: 0.0


